In [1]:
import pandas as pd
import geopandas as gpd
import os


In [2]:
folder = r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data"
file_name = r"washington_rentals.csv"
geocoded_file_name = f"geocoded_{file_name}"

dots_source = pd.read_csv(f"{folder}\{file_name}")

gdf = gpd.GeoDataFrame(
    dots_source, geometry=gpd.points_from_xy(dots_source.Longitude, dots_source.Latitude), crs="EPSG:4326"
)

In [8]:
# demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source\tl_2023_53_bg.shp")

demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")
demographic_areas.to_crs("EPSG:4326", inplace=True)

demographic_areas["GEOID"] = \
    demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)  \
    + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3) \
    + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6) \
    + demographic_areas["BLOCKGROUP_FIPS"].astype(str)

demographic_areas = demographic_areas[["GEOID", "geometry"]].rename(columns={"GEOID":"cbg_geoid"})

cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
cbsa_source.to_crs("EPSG:4326", inplace=True)
cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID":"cbsa_geoid", "NAME": "cbsa_name"})

state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
state_source.to_crs("EPSG:4326", inplace=True)
state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID":"state_id", "State_Name": "state_name"})


In [9]:
geocoded_dots = gdf.sjoin(demographic_areas, how = "left").drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(cbsa_source, how = 'left').drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(state_source, how = 'left').drop(["index_right"], axis=1)

geocoded_dots = geocoded_dots.drop(['geometry', 'Unnamed: 0'],axis=1)
geocoded_dots.to_csv(f"{folder}/{geocoded_file_name}", index= False) 

In [10]:
display(geocoded_dots)

,Property ID,URL,Property Type,Address,City,State,ZIP Code,Country Code,Latitude,Longitude,...,Max Beds,Max Baths,Max Square Feet,Max Rent Price,Description,cbg_geoid,cbsa_geoid,cbsa_name,state_id,State_Code
0,22700195,/WA/Auburn/Watermark/apartment/22700195,5,31655 Military Rd S,Auburn,WA,98001.0,1,47.318273,-122.285212,...,4,2.0,1342.0,2467.0,Seize your chance to live in a brand new commu...,530330299012,42660,"Seattle-Tacoma-Bellevue, WA",48.0,WA
1,209269,/WA/Auburn/4903-S-283rd-Pl-98001/home/209269,6,4903 S 283rd Pl,Auburn,WA,98001.0,1,47.347206,-122.271890,...,4,3.0,2650.0,4295.0,Welcome to this beautiful home in the highly d...,530330298063,42660,"Seattle-Tacoma-Bellevue, WA",48.0,WA
2,40265097,/WA/Auburn/27810-47th-Pl-S-98001/home/40265097,6,27810 47th Pl S,Auburn,WA,98001.0,1,47.352111,-122.274841,...,3,2.5,1930.0,3147.0,Offered By: Spinnaker Property Management Desc...,530330298063,42660,"Seattle-Tacoma-Bellevue, WA",48.0,WA
3,113179480,/WA/Auburn/3071-S-276th-Ct-98001/home/113179480,6,3071 S 276th Ct,Auburn,WA,98001.0,1,47.353470,-122.293626,...,4,2.5,2623.0,4200.0,Avail Now. And just in time to enjoy summer wi...,530330298061,42660,"Seattle-Tacoma-Bellevue, WA",48.0,WA
4,173657113,/WA/Auburn/5638-S-302nd-St-98001/home/173657113,6,5638 S 302nd St,Auburn,WA,98001.0,1,47.331213,-122.264241,...,4,2.5,2472.0,3750.0,Welcome Home! This Space is sprawling with lig...,530330299021,42660,"Seattle-Tacoma-Bellevue, WA",48.0,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861,126759534,/WA/Walla-Walla/1987-Alco-Ave-99362/home/12675...,4,1987 Alco Ave,Walla Walla,WA,99362.0,1,46.044910,-118.339810,...,3,2.0,1080.0,1795.0,NaN,530719206004,47460,"Walla Walla, WA",48.0,WA
4862,126555038,/WA/Walla-Walla/1800-Evergreen-St-99362/apartm...,5,1800 Evergreen St,Walla Walla,WA,99362.0,1,46.073318,-118.308702,...,2,1.0,712.0,1150.0,NaN,530719208012,47460,"Walla Walla, WA",48.0,WA
4863,126381245,/WA/Walla-Walla/536-Pleasant-St-99362/home/126...,6,536 Pleasant St,Walla Walla,WA,99362.0,1,46.061948,-118.323622,...,4,2.0,1725.0,2495.0,Charming 4 bedroom Home<BR><BR>Welcome to this...,530719207022,47460,"Walla Walla, WA",48.0,WA
4864,190574084,/WA/Walla-Walla/944-N-9th-Ave-99362/apartment/...,5,944 N 9th Ave,Walla Walla,WA,99362.0,1,46.074667,-118.350879,...,0,1.0,6410.0,5950.0,FOR LEASE; commercial/light industrial propert...,530719205001,47460,"Walla Walla, WA",48.0,WA
